# The story so far

Last week, we saw how we can write command-line programs that take arguments. We saw:

- Arguments are parsed largely by the shell
- We get the results of that parsing in a list of strings (`sys.argv`)
- We can use `argparse` to interpret `sys.argv` on our behalf
    - Types
    - (including files)
    - Required
    - Keyword arguments
    - Defaults
    - Even accept a number of values for the same argument!

All of this contributes to a healthy command-line program. Typically, that kind of program (like many command-line utilities) runs with the inputs from the arguments and then does it computation and then prints some results.

# Today, we're going to get deeper and more serious

1. We're going to look at the `Cmd` module in Python's standard library, which is meant for interactive command-line programs.
2. We're going to look at how `Cmd` gives us some flexibility in interpreting the user's inputs
3. Then we're going to write a module with extensions that add new commands to our Cmd-powered system
4. Then we're see how we can load *multiple* modules with extensions
5. Then we'll tie it together, using `argparse` to specify which extentions should be loaded

Along the way, we're going to learn lots of stuff:
- Object-oriented programming
- Modules and how they're built, and how they're imported
- Class attributes and methods vs. functions

# What is `Cmd`?

It's a module in the Python standard library. The standard library contains hundreds of modules and packages that come with Python when you download and install it. These modules are guaranteed to be installed anywhere Python is installed.

Just because a module is in the standard library doesn't mean it is automatically loaded into memory! Only "builtin" modules are loaded automatically, as well as some "frozen" modules. You need to use `import` to load a standard-library module, unless it's builtin/frozen.

# What's the `if __name__ == '__main__'` line doing?

1. When we import a module, or when we run a program, the entire thing is executed from start to finish.
2. The `__name__` variable is always defined. It contains a string value, which will be one of two things:
    - The string `'__main__'`, which means that it was the first program/module to run, not imported by someone else
    - The string corresponding to the module's name, e.g., `mycmd`, which indicates that the file was imported by someone else
3. Because the entire file is executed, and because `__name__` has this two-value potential, we can use `if` to determine whether it's running as the main program and then run something appropriate.

This allows us to have modules that are both modules (importable) and programs (runnable). This line distinguishes between what happepns when we import (above the line) and what happens when we run (the entire file).

# How do we add commands to our Cmd-based system?

Inside of the class that inherits from `cmd.Cmd`, we add methods. Each method must have a name that starts with `do_`, and then has any word you want. If you define `do_xyz`, then the command `xyz` will exist. And if you type `xyz abc` on the command line in the program, then `do_xyz` will be invoked, and `abc` will be passed as its argument.

# How do you set up an exit?

The standard way to do this is to define a `do_EOF` method. That returns `True`, and that exits from the loop.

# Exercise 7: Simple calculator

1. Using `Cmd`, write a very simple calculator program. The user should be able to type `add x y`, where `x` and `y` are both numbers, and prints the sum of those two.
2. Repeat this for `mul` and `sub` and `div`, to multiply, subtract, and divide.
3. Let's assume for now that the user will enter numbers where needed.

# Next up

1. Additional `Cmd` functionality to "DRY up" our code
    - Reduce repetition
    - Handle more interesting (e.g., symbols) in our calculator
2. Dynamically adding new methods to our calculator

Resume at :08

# DRY -- "don't repeat yourself"

- If you have the same code repeated several lines in a row, then use a loop
- If you have the same code in several parts of your program, then use a function
- If you have the same code in several programs, then use a library/module/package

This has a lot of advantages -- the biggest one being that you save time! You write things once and then use them many times.

# Exercise 8: Documented, DRY-ed up calculator

1. Write a method, `to_integers`, that takes the input string (`line`) and returns a two-element list of integers based on the input string.
2. (You can print an error message if you want, but we're not really going to deal with error handling here.)
3. Modify your `do_` methods to use `to_integers` so that teach method is a bit shorter to write/update/understand.
4. Update each of these methods such that they have docstrings, and are documented from within Cmd.

# What if I want to mess with the input line?

Right now, if someone enters `ABC` and we have `do_abc`, it won't match.  In many cases, we might want to modify the input line slightly before it is passed to a `do_X` method.

The way to do this is with `precmd`. If you define a method by this name in your class, then whatever the user enters is first passed to `precmd`.  This means that you can mess with the line -- whatever `precmd` returns is then actually parsed and handed over to a `do_X` method.

# Exercise 9: Allow for operators

Modify your calculator program, such that instead of saying `add`, we can say `+`. Ditto for the other 3 operators (`-`, `*`, `/`).

If someone enters `+ 2 3`, then it should invoke `do_add` on `2 3` and then go through the transformation.

# Next up

1. Methods and adding them dynamically
2. Modules, loading them, and adding dynamic methods

Resume at :04

# What is a method?

You're typically told:

- A method is basically a function defined inside of a class
- You invoke the method on the instance, and the instance is assigned to the first parameter, `self`
- If you want, you can invoke the method directly on the class, passing the instance as the first argument

It's true that methods are rewritten magically.

In [1]:
s = 'abcde'

s.upper()

'ABCDE'

When we invoke `s.upper()`, Python actually rewrites this to be

In [3]:
str.upper(s)  # the instance became the first argument!

'ABCDE'

In [4]:
help(str.upper)

Help on method_descriptor:

upper(self, /) unbound builtins.str method
    Return a copy of the string converted to uppercase.



How does Python perform this kind of switcheroo? When does it do it?

How are methods and functions actually different from one another?

When you define a method inside of a class, two things happen:

- The function you defined is kept, as is, but not where you might think! It's stored inside of a class attribute called `__dict__`.
- The name that you gave to your function is actually assigned to a special "method" object. That acts as a wrapper, and looks to see how you invoked it, and whether it needs to shuffle around the arguments.

In [5]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

In [6]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '__init__': <function __main__.Person.__init__(self, name)>,
              'greet': <function __main__.Person.greet(self)>,
              '__static_attributes__': ('name',),
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [7]:
type(Person.__init__)

function

In [8]:
type(Person.greet)

function

# Class attributes

A class attribute is an attribute on a class. We can access a class attribute in at least two ways:

- Directly, via the class
- Indirectly, via an instance of that class. If we ask for the attribute on the instance, and it isn't there, then Python turns to the class and asks if there's an attribute of that name. If so, then we get back that class attribute.

If the class attribute is a "descriptor" in Python, then it has the opportunity to see that we are requesting it via the instance, and return a different value and/or invoke different functionality.

With a method:

- If we retrieve/invoke it via the class, then we get the original function, unchanged.

In [14]:
# retrieve a method via the class, and we just get the function back that we originally defined
Person.__init__ is Person.__dict__['__init__']

True

In [15]:
# retrieve a method via the class, and we just get the function back that we originally defined
Person.greet is Person.__dict__['greet']

True

In [16]:
p = Person('Reuven')

In [17]:
p.__init__ is Person.__dict__['__init__']

False

In [18]:
p.greet is Person.__dict__['greet']

False

In [19]:
type(p.greet)

method

When we retrieve `Person.greet`, that triggers the descriptor, which is activated when we retrieve that attribute. It sees that we're requesting this class attribute via the class, and returns the original function value.

BUT when we retrieve `p.greet`, that triggers the descriptor, which is activated when we retrieve that attribute. It sees that we're requesting the class attribute via an instance, and returns the method object. That method object then performs the rewriting, such that the instance is assigned to `self`, the first parameter.

# Who cares?

Why does this matter to us at all?

- Because every time you invoke a method in Python, especially on the instance, descriptors are used to rewrite the arguments, moving the instance into `self`. This is why every method needs a `self` parameter.
- If you want to add a new method to a class at runtime, it's really important to understand that we aren't just defining a function. Rather, we are going to be adding a *method*, and that's not quite the same thing.